In [71]:
!pwd
!ls

/d/GH/GitWorkSpace/bank_model_competiton/data/v7
model_stacking_v7.ipynb
process_v7.ipynb
test.dat.v7
train.dat.v7


In [61]:
import pandas as pd 
import matplotlib.pyplot as plt
import statistics
import datetime
import seaborn as sns
import os
import numpy as np
import time 
from sklearn.preprocessing import OrdinalEncoder


plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

suffix = os.path.split(os.getcwd())[-1]

root_dir='../../'

train_path=root_dir+'train.csv'
train_tx_path=root_dir+'train_bank_statement.csv'
test_path=root_dir+'testaa.csv'
test_tx_path=root_dir+'testaa_bank_statement.csv'

print(suffix)
print('process time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))

v7
process time :  2025-08-27 18:43:32


In [62]:
mode="test"

NROWS = 100000000000000

if mode == "train":
    base_input_path = train_path
    tx_input_path = train_tx_path
    output_path = "train.dat.%s" % suffix
else:
    base_input_path = test_path
    tx_input_path = test_tx_path
    output_path = "test.dat.%s" % suffix

print(mode, base_input_path, tx_input_path, output_path)

test ../../testaa.csv ../../testaa_bank_statement.csv test.dat.v7


#  基础特征

In [63]:
!pwd
df = pd.read_csv(base_input_path, index_col=['id'], nrows=NROWS)
print(df.head())
print(df.shape)

df['balance_account_avg'] = df['balance'] / df['balance_accounts']
df['loan_term_avg'] = df['loan'] / df['term']
df['balance_accounts_ratio'] = df['balance_accounts'] / df['total_accounts']
df['interest_rate_log'] = df['interest_rate'].apply(lambda x : np.log(x))

interest_rate_labels = ['%d' % i for i in range(20)]
print(interest_rate_labels)
df['interest_rate_cut'] = pd.cut(df['interest_rate'], bins=20, labels = interest_rate_labels)

#level 特征
df['level_hash'] = df['level'].apply(lambda x : hash(x) % 1000 )

encoder = OrdinalEncoder(categories=[['A0','A1','A2','A3','A4','A5','B0','B1','B2','B3','B4','B5','C0','C1','C2','C3','C4','C5','D0','D1','D2','D3','D4','D5','E0','E1','E2','E3','E4','E5']]) 
df['level_ord'] = encoder.fit_transform(df[['level']].values)

df

/d/GH/GitWorkSpace/bank_model_competiton/data/v7
       title  career  zip_code  residence   loan  term  interest_rate  \
id                                                                      
53480     10     5.0    512068          0   5950    36          14.10   
53481      8    10.0    401852          2  15350    36           5.25   
53482      8    10.0    321693          0  11050    36          13.65   
53483      8     7.0    231397          0  15350    36          14.20   
53484      7     7.0    171193          0   7800    36          10.00   

       issue_time  syndicated  installment  record_time  history_time  \
id                                                                      
53480  1054600001           0            0   1054600080     795400642   
53481  1172880047           0            0   1172882338     756003297   
53482  1238631003           0            0   1277511093     658715225   
53483  1136161107           0            0   1198371199     774316843   
5

,title,career,zip_code,residence,loan,term,interest_rate,issue_time,syndicated,installment,...,balance_limit,balance,level,balance_account_avg,loan_term_avg,balance_accounts_ratio,interest_rate_log,interest_rate_cut,level_hash,level_ord
id,,,,,,,,,,,,,,,,,,,,,
53480,10,5.0,512068,0,5950,36,14.10,1054600001,0,0,...,10100.0,8233.00,B0,914.777778,165.277778,0.529412,2.646175,6,147,6.0
53481,8,10.0,401852,2,15350,36,5.25,1172880047,0,0,...,58600.0,23460.00,A0,2606.666667,426.388889,0.529412,1.658228,0,483,0.0
53482,8,10.0,321693,0,11050,36,13.65,1238631003,0,0,...,15500.0,7853.00,B1,872.555556,306.944444,0.529412,2.613740,6,860,7.0
53483,8,7.0,231397,0,15350,36,14.20,1136161107,0,0,...,10100.0,5815.00,B2,646.111111,426.388889,0.529412,2.653242,6,604,8.0
53484,7,7.0,171193,0,7800,36,10.00,1214960354,0,1,...,13100.0,8225.00,A3,913.888889,216.666667,0.529412,2.302585,3,965,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73529,0,8.0,601107,1,10000,12,18.85,1130976000,0,0,...,3818.0,2224.69,A4,741.563333,833.333333,0.500000,2.936513,10,690,4.0
73530,0,10.0,601102,1,10000,12,29.30,1156204800,0,0,...,5502.0,4126.71,B4,687.785000,833.333333,1.000000,3.377588,18,896,10.0
73531,0,4.0,601408,1,11000,12,24.75,1144108800,0,0,...,4844.0,2710.96,A3,903.653333,916.666667,0.375000,3.208825,15,965,3.0


#  交易特征处理

In [64]:
df_tx = pd.read_csv(tx_input_path, nrows = NROWS, index_col=['id'])
df_tx['time_format'] = df_tx['time'].apply(lambda x: datetime.datetime.fromtimestamp(x))

print(df['record_time'])
print(df_tx)

# 假设 ：issue time 是记录违约的时间，那只能用在此之前的流水
# df_tx_merge = pd.merge(df_tx, df[['record_time','issue_time']], on = 'id', how='left')
# df_tx = df_tx_merge[ df_tx_merge['time'] <  df_tx_merge['issue_time'] ]

id
53480    1054600080
53481    1172882338
53482    1277511093
53483    1198371199
53484    1214957804
            ...    
73529    1125964800
73530    1157068800
73531    1111622400
73532    1116892800
73533    1146614400
Name: record_time, Length: 20054, dtype: int64
             time  direction       amount         time_format
id                                                           
56693  1144195200          1   542.210000 2006-04-05 08:00:00
56693  1145145600          1   283.490000 2006-04-16 08:00:00
56693  1145145600          0  5604.080000 2006-04-16 08:00:00
56693  1146787200          1   539.380000 2006-05-05 08:00:00
56693  1147737600          0  5951.750000 2006-05-16 08:00:00
...           ...        ...          ...                 ...
71870  1160956800          1   493.403945 2006-10-16 08:00:00
71870  1161043200          0     9.462382 2006-10-17 08:00:00
71870  1161388800          1   222.936072 2006-10-21 08:00:00
71870  1161475200          0   222.936072 2006-1

In [65]:
#days_diff, tm_count, total_amount, amount_1, amount_0, total_amount_day_avg, amount_1_day_avg, amount_0_day_avg
print(df_tx.shape)

df_tx_stat = pd.DataFrame()
df_tx_stat['tx_tmstp_max'] = df_tx.groupby('id')['time'].agg('max')
df_tx_stat['tx_tmstp_min'] = df_tx.groupby('id')['time'].agg('min')

df_tx_stat['tx_time_max'] = df_tx.groupby('id')['time_format'].agg('max')
df_tx_stat['tx_time_min'] = df_tx.groupby('id')['time_format'].agg('min')

df_tx_stat['tx_max_min_days'] = (df_tx_stat['tx_time_max'] - df_tx_stat['tx_time_min'])
df_tx_stat['tx_max_min_days'] = df_tx_stat['tx_max_min_days'].map(lambda x : x.days)

df_tx_stat['tx_count'] = df_tx.groupby('id')['amount'].agg('count')
df_tx_stat['total_amount'] = df_tx.groupby('id')['amount'].agg('sum')

df_tx_stat['1_amount'] = df_tx.groupby('id').apply( lambda x : x [ x['direction'] == 1]['amount'].sum())
df_tx_stat['0_amount'] = df_tx.groupby('id').apply( lambda x : x [ x['direction'] == 0]['amount'].sum())

df_tx_stat['total_amount_avg']  = df_tx_stat['total_amount'] / df_tx_stat['tx_max_min_days'] 
df_tx_stat['1_amount_avg']  = df_tx_stat['1_amount'] / df_tx_stat['tx_max_min_days'] 
df_tx_stat['0_amount_avg']  = df_tx_stat['0_amount'] / df_tx_stat['tx_max_min_days'] 

df_tx_stat['total_amount_avg2']  = df_tx_stat['total_amount'] / df_tx_stat['tx_count'] 
df_tx_stat['1_amount_avg2']  = df_tx_stat['1_amount'] / df_tx_stat['tx_count'] 
df_tx_stat['0_amount_avg2']  = df_tx_stat['0_amount'] / df_tx_stat['tx_count'] 

df_tx_stat.drop([ 'tx_time_max', 'tx_time_min'], axis = 1, inplace=True)
df_tx_stat

(646666, 4)


,tx_tmstp_max,tx_tmstp_min,tx_max_min_days,tx_count,total_amount,1_amount,0_amount,total_amount_avg,1_amount_avg,0_amount_avg,total_amount_avg2,1_amount_avg2,0_amount_avg2
id,,,,,,,,,,,,,
53480,1053302400,1039046400,165,46,61815.520000,11245.520000,50570.000000,374.639515,68.154667,306.484848,1343.815652,244.467826,1099.347826
53481,1170806400,1158364800,144,56,38555.840000,20161.940000,18393.900000,267.748889,140.013472,127.735417,688.497143,360.034643,328.462500
53482,1275350400,1260489600,172,74,81589.800000,44977.460000,36612.340000,474.359302,261.496860,212.862442,1102.564865,607.803514,494.761351
53483,1195776000,1182038400,159,73,48205.320000,24048.520000,24156.800000,303.178113,151.248553,151.929560,660.346849,329.431781,330.915068
53484,1214438400,1199404800,174,91,78113.610000,29797.860000,48315.750000,448.928793,171.252069,277.676724,858.391319,327.449011,530.942308
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73515,1161475200,1153353600,94,25,9477.597620,5109.004148,4368.593472,100.825507,54.351108,46.474399,379.103905,204.360166,174.743739
73517,1161475200,1145923200,180,272,212302.562407,126112.342194,86190.220212,1179.458680,700.624123,478.834557,780.524126,463.648317,316.875810
73522,1161475200,1145923200,180,116,45701.276358,24957.995111,20743.281247,253.895980,138.655528,115.240451,393.976520,215.155130,178.821390


In [66]:
df_tx_stat.shape

(8180, 13)

# 合并交易特征

In [67]:
df_concat = pd.concat([df, df_tx_stat], axis=1)
print(df_concat.shape)
df_concat.reset_index(inplace=True)

(20054, 37)


In [68]:
df_concat.rename(columns={'index':'id'}, inplace=True)
print(df_concat)
df_concat.to_csv(output_path, index=False)
print('output_path : ', output_path)
print('process time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))
!ls

          id  title  career  zip_code  residence   loan  term  interest_rate  \
0      53480     10     5.0    512068          0   5950    36          14.10   
1      53481      8    10.0    401852          2  15350    36           5.25   
2      53482      8    10.0    321693          0  11050    36          13.65   
3      53483      8     7.0    231397          0  15350    36          14.20   
4      53484      7     7.0    171193          0   7800    36          10.00   
...      ...    ...     ...       ...        ...    ...   ...            ...   
20049  73529      0     8.0    601107          1  10000    12          18.85   
20050  73530      0    10.0    601102          1  10000    12          29.30   
20051  73531      0     4.0    601408          1  11000    12          24.75   
20052  73532      0     3.0    601904          1   8000    12          22.00   
20053  73533      2     1.0    601809          1   7000    12          14.95   

       issue_time  syndicated  ...  tx_

In [69]:
!wc -l $output_path

20055 test.dat.v7


In [70]:
print('done  time : ',time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime()))
print(df_concat.columns, '\n', df_concat.shape)
df_concat

done  time :  2025-08-27 18:43:42
Index(['id', 'title', 'career', 'zip_code', 'residence', 'loan', 'term',
       'interest_rate', 'issue_time', 'syndicated', 'installment',
       'record_time', 'history_time', 'total_accounts', 'balance_accounts',
       'balance_limit', 'balance', 'level', 'balance_account_avg',
       'loan_term_avg', 'balance_accounts_ratio', 'interest_rate_log',
       'interest_rate_cut', 'level_hash', 'level_ord', 'tx_tmstp_max',
       'tx_tmstp_min', 'tx_max_min_days', 'tx_count', 'total_amount',
       '1_amount', '0_amount', 'total_amount_avg', '1_amount_avg',
       '0_amount_avg', 'total_amount_avg2', '1_amount_avg2', '0_amount_avg2'],
      dtype='object') 
 (20054, 38)


,id,title,career,zip_code,residence,loan,term,interest_rate,issue_time,syndicated,...,tx_count,total_amount,1_amount,0_amount,total_amount_avg,1_amount_avg,0_amount_avg,total_amount_avg2,1_amount_avg2,0_amount_avg2
0,53480,10,5.0,512068,0,5950,36,14.10,1054600001,0,...,46.0,61815.52,11245.52,50570.00,374.639515,68.154667,306.484848,1343.815652,244.467826,1099.347826
1,53481,8,10.0,401852,2,15350,36,5.25,1172880047,0,...,56.0,38555.84,20161.94,18393.90,267.748889,140.013472,127.735417,688.497143,360.034643,328.462500
2,53482,8,10.0,321693,0,11050,36,13.65,1238631003,0,...,74.0,81589.80,44977.46,36612.34,474.359302,261.496860,212.862442,1102.564865,607.803514,494.761351
3,53483,8,7.0,231397,0,15350,36,14.20,1136161107,0,...,73.0,48205.32,24048.52,24156.80,303.178113,151.248553,151.929560,660.346849,329.431781,330.915068
4,53484,7,7.0,171193,0,7800,36,10.00,1214960354,0,...,91.0,78113.61,29797.86,48315.75,448.928793,171.252069,277.676724,858.391319,327.449011,530.942308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20049,73529,0,8.0,601107,1,10000,12,18.85,1130976000,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20050,73530,0,10.0,601102,1,10000,12,29.30,1156204800,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20051,73531,0,4.0,601408,1,11000,12,24.75,1144108800,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20052,73532,0,3.0,601904,1,8000,12,22.00,1163808000,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
